In [1]:
# 读取训练数据, 训练集包含500个模型结构，以及这些结构在cplfw，market1501，dukemtmc等8个任务上的性能排序
import json
with open('./data/CVPR_2022_NAS_Track2_train.json', 'r') as f:
    train_data = json.load(f)
print(train_data['arch1'])
print('train_num:',len(train_data.keys()))

{'cplfw_rank': 9, 'market1501_rank': 361, 'dukemtmc_rank': 426, 'msmt17_rank': 433, 'veri_rank': 327, 'vehicleid_rank': 480, 'veriwild_rank': 425, 'sop_rank': 367, 'arch': 'l231131331121121331111211121331321321'}
train_num: 500


In [2]:
import pandas as pd
import numpy as np

In [3]:
target_cols = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']

In [4]:
def get_df(train_data):
    ret = []
    for k, v in train_data.items():
        tmp = list(v['arch'])
        tmp1 = []
        for c in target_cols:
            tmp1.append(v[c])
        ret.append(tmp+tmp1+[k,v['arch']])
    retf = pd.DataFrame(ret,columns=[f'col{_}' for _ in range(len(tmp))]+target_cols+['id','arch'])
    retf['col0'] = retf['col0'].map({'l':1, 'j':2, 'k':3})
    int_cols = [x for x in retf.columns if x not in ['id','arch']]
    retf[int_cols] = retf[int_cols].astype(float)
    return retf

In [5]:
train_df = get_df(train_data)

In [6]:
for c in target_cols:
    train_df[c+'_log1p'] = train_df[c].map(lambda x: np.log1p(x))
    
    bins = train_df[c].quantile(np.arange(0.0,1.05,0.05)).values
    def quantile(x,bins=bins):
        for i in bins[1:]:
            if x<=i:
                return i
    train_df[c+'_quantile'] = train_df[c].map(quantile)

In [7]:
train_df.columns

Index(['col0', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8',
       'col9', 'col10', 'col11', 'col12', 'col13', 'col14', 'col15', 'col16',
       'col17', 'col18', 'col19', 'col20', 'col21', 'col22', 'col23', 'col24',
       'col25', 'col26', 'col27', 'col28', 'col29', 'col30', 'col31', 'col32',
       'col33', 'col34', 'col35', 'col36', 'cplfw_rank', 'market1501_rank',
       'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank',
       'veriwild_rank', 'sop_rank', 'id', 'arch', 'cplfw_rank_log1p',
       'cplfw_rank_quantile', 'market1501_rank_log1p',
       'market1501_rank_quantile', 'dukemtmc_rank_log1p',
       'dukemtmc_rank_quantile', 'msmt17_rank_log1p', 'msmt17_rank_quantile',
       'veri_rank_log1p', 'veri_rank_quantile', 'vehicleid_rank_log1p',
       'vehicleid_rank_quantile', 'veriwild_rank_log1p',
       'veriwild_rank_quantile', 'sop_rank_log1p', 'sop_rank_quantile'],
      dtype='object')

In [8]:
def get_step(train_df):
    res0 = []
    res1 = []
    res2 = []
    res3 = []
    res4 = []
    map_={'l':1, 'j':2, 'k':3}
    time_step=12
    for item in train_df.arch:
        ret = np.array(list(item[1:]),dtype=np.float32).reshape(-1,3)
        res0.append(ret[:,0][:time_step])
        res1.append(ret[:,1][:time_step])
        res2.append(ret[:,2][:time_step])
        res3.append(np.array([map_[item[0]]]*time_step))
        res4.append([map_[item[0]]]+list(np.array(list(item[1:]),dtype=np.float32)))
        
    train_df['head'] = res0
    train_df['mlp'] = res1
    train_df['emb'] = res2
    train_df['depth'] = res3
    train_df['all_emb'] = res4

In [9]:
#     retf['col0'] = retf['col0'].map({'j':10, 'k':11, 'l':12})
#     retf[num_heads] = retf[num_heads].replace({1:12, 2:11, 3:10})
#     retf[mlp_ratio] = retf[mlp_ratio].replace({1:4, 2:3.5, 3:3})
#     retf[emb_dim] = retf[emb_dim].replace({1:768})

In [10]:
def get_step(train_df):
    res0 = []
    res1 = []
    res2 = []
    res3 = []
    res4 = []
    map_={'l':1, 'j':2, 'k':3}
    time_step=12
    head_ = set([1+_*3 for _ in range(12)])
    mlp_ = set([2+_*3 for _ in range(12)])
    emb_ = set([3+_*3 for _ in range(12)])
    depth_ = set([0])
    for item in train_df.arch:
        ret = np.array(list(item[1:]),dtype=np.float32).reshape(-1,3)
        res0.append([1 if x in head_ else 0 for x in range(37)])
        res1.append([1 if x in mlp_ else 0 for x in range(37)])
        res2.append([1 if x in emb_ else 0 for x in range(37)])
        res3.append([1 if x in depth_ else 0 for x in range(37)])
        res4.append([map_[item[0]]]+list(np.array(list(item[1:]),dtype=np.float32)))
        
    train_df['head'] = res0
    train_df['mlp'] = res1
    train_df['emb'] = res2
    train_df['depth'] = res3
    train_df['all_emb'] = res4

In [11]:
get_step(train_df)

In [12]:
def get_step_0503(train_df):
    res0 = []
    res1 = []
    res2 = []
    res3 = []
    res4 = []
    map_={'l':.1, 'j':.2, 'k':.3}
    time_step=12
    head_ = set([1+_*3 for _ in range(12)])
    mlp_ = set([2+_*3 for _ in range(12)])
    emb_ = set([3+_*3 for _ in range(12)])
    depth_ = set([0])
    for item in train_df.arch:
        ret = np.array(list(item[1:]),dtype=np.float32).reshape(-1,3)
        res0.append([int(item[x])/10 if x in head_ else 0 for x in range(37)])
        res1.append([int(item[x])/10 if x in mlp_ else 0 for x in range(37)])
        res2.append([int(item[x])/10 if x in emb_ else 0 for x in range(37)])
        res3.append([map_[item[0]]]+[0]*36)
        res4.append([map_[item[0]]]+list(np.array(list(item[1:]),dtype=np.float32)/10))
#         tr=[]
#         for k,v in enumerate(item):
#             if k>0 and k not in emb_:
#                 tr.append(np.float32(v))
                
#         res4.append([map_[item[0]]]+tr)
        
    train_df['head'] = res0
    train_df['mlp'] = res1
    train_df['emb'] = res2
    train_df['depth'] = res3
    train_df['all_emb'] = res4

In [13]:
# get_step_0503(train_df)

In [14]:
len(train_df.all_emb.iloc[0])

37

In [15]:
# for c in ['head','mlp']:
#     train_df[c+'_diff1'] = train_df[c].map(lambda x: x - np.concatenate([np.array([0]),x[:-1]]))
#     train_df[c+'_diff2'] = train_df[c].map(lambda x: x - np.concatenate([np.array([0,0]),x[:-2]]))

In [16]:
from sklearn.model_selection import KFold
from tqdm import tqdm_notebook as tqdm

In [17]:
#定义数据
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self,df,use_cols,target_cols,show=0):
        self.df = df
        self.show = show
        self.use_cols = use_cols
        self.target_cols = target_cols

        self.prepare_data()
        
    def __len__(self):
        return self.df.shape[0]
    
    def prepare_data(self):
        self.y = self.df[self.target_cols].values
        self.y_quantile = self.df[[_+'_quantile' for _ in self.target_cols]].values
        self.y0 = self.df[[self.target_cols[0]]].values
        self.y1 = self.df[[self.target_cols[1]]].values
        self.y2 = self.df[[self.target_cols[2]]].values
        self.y3 = self.df[[self.target_cols[3]]].values
        self.y4 = self.df[[self.target_cols[4]]].values
        self.y5 = self.df[[self.target_cols[5]]].values
        self.y6 = self.df[[self.target_cols[6]]].values
        self.y7 = self.df[[self.target_cols[7]]].values
        
#         self.inputs = self.df[use_cols].values
        uc = ['all_emb']
#         uc = ['head','mlp','emb']
        tmp_dt = {}
        for c in uc:
            tmp_dt[c] = np.array(self.df[c].tolist())
        self.inputs = np.concatenate([tmp_dt[c][:, None] for c in uc], 1).transpose(0, 2, 1)
        
#         uc= ['all_emb','depth','head','mlp','emb']
#         tmp_dt = {}
#         for c in uc:
#             tmp_dt[c] = np.array(self.df[c].tolist())
#         self.inputs1= np.concatenate([tmp_dt[c][:, None] for c in uc], 1).transpose(0, 2, 1)
        
        
        if self.show==1:
            print('inputs_shape',self.inputs.shape)
#             print('inputs1_shape',self.inputs1.shape)
            print('y_shape',self.y.shape)
            print('y_quantile',self.y_quantile.shape)
        
    def __getitem__(self, idx):
        data = {
            "input": torch.tensor(self.inputs[idx], dtype=torch.float),
#             "input1": torch.tensor(self.inputs1[idx], dtype=torch.float),
            "y": torch.tensor(self.y[idx], dtype=torch.float),
            "y_quantile": torch.tensor(self.y_quantile[idx], dtype=torch.float),
            "y0": torch.tensor(self.y0[idx], dtype=torch.float),
            "y1": torch.tensor(self.y1[idx], dtype=torch.float),
            "y2": torch.tensor(self.y2[idx], dtype=torch.float),
            "y3": torch.tensor(self.y3[idx], dtype=torch.float),
            "y4": torch.tensor(self.y4[idx], dtype=torch.float),
            "y5": torch.tensor(self.y5[idx], dtype=torch.float),
            "y6": torch.tensor(self.y6[idx], dtype=torch.float),
            "y7": torch.tensor(self.y7[idx], dtype=torch.float),
        }
        
        return data

In [18]:
train_df.columns

Index(['col0', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8',
       'col9', 'col10', 'col11', 'col12', 'col13', 'col14', 'col15', 'col16',
       'col17', 'col18', 'col19', 'col20', 'col21', 'col22', 'col23', 'col24',
       'col25', 'col26', 'col27', 'col28', 'col29', 'col30', 'col31', 'col32',
       'col33', 'col34', 'col35', 'col36', 'cplfw_rank', 'market1501_rank',
       'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank',
       'veriwild_rank', 'sop_rank', 'id', 'arch', 'cplfw_rank_log1p',
       'cplfw_rank_quantile', 'market1501_rank_log1p',
       'market1501_rank_quantile', 'dukemtmc_rank_log1p',
       'dukemtmc_rank_quantile', 'msmt17_rank_log1p', 'msmt17_rank_quantile',
       'veri_rank_log1p', 'veri_rank_quantile', 'vehicleid_rank_log1p',
       'vehicleid_rank_quantile', 'veriwild_rank_log1p',
       'veriwild_rank_quantile', 'sop_rank_log1p', 'sop_rank_quantile', 'head',
       'mlp', 'emb', 'depth', 'all_emb'],
      dtype='object')

In [19]:
train0 = train_df.sample(400, random_state=666).reset_index(drop=True)
val0 = train_df[~train_df.id.isin(train0.id)].reset_index(drop=True)

In [20]:
use_cols = [x for x in train_df.columns if 'col' in x]

In [21]:
len(use_cols)

37

In [22]:
train_dataset = MyDataset(train0, use_cols, target_cols, 1)
val_dataset = MyDataset(val0, use_cols, target_cols, 1)

inputs_shape (400, 37, 1)
y_shape (400, 8)
y_quantile (400, 8)
inputs_shape (100, 37, 1)
y_shape (100, 8)
y_quantile (100, 8)


In [23]:
import gc
import os
import time
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup


def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    
def count_parameters(model, all=False):
    if all:
        return sum(p.numel() for p in model.parameters())
    else:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)


def save_model_weights(model, modelpath, filename):
    torch.save(model.state_dict(), modelpath+filename)
    return f"\n -> Save weights to {modelpath+filename}\n"

In [24]:
import scipy

In [25]:
def compute_metric(pred, y):
    corr = []
    if pred.shape[1]>2:
        for i in range(8):
            corr.append(scipy.stats.stats.kendalltau(pred[:, i], y[:, i])[0])
    else:
        corr.append(scipy.stats.stats.kendalltau(pred, y)[0])
    return np.array(corr)

class CVPRLoss(nn.Module):
    def __call__(self, pred, y):
        pred_mean, y_mean = pred.mean(0), y.mean(0)
        pred_std, y_std = pred.std(0), y.std(0)
        corr = ((pred - pred_mean) * (y - y_mean)).sum(0) / ((((pred - pred_mean)**2).sum(0).sqrt()) * (((y - y_mean)**2).sum(0).sqrt())) 
    
#         cos = nn.CosineSimilarity(dim=0, eps=1e-08)
#         corr1 = cos(pred, y)
#         corr2 = (corr+corr1)/2
        return 1-corr

from fast_soft_sort.pytorch_ops import soft_rank
class CVPRLoss1(nn.Module):
    #spearman
    def __call__(self, pred, y):
        pred = pred.cpu()
        y = y.cpu()
        return 1-torch.cat(
            [self.spearman(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,1) for i in range(y.shape[1])]
        ).reshape(1, -1)
        
    def corrcoef(self, target, pred):
        # np.corrcoef in torch from @mdo
        # https://forum.numer.ai/t/custom-loss-functions-for-xgboost-using-pytorch/960
        pred_n = pred - pred.mean()
        target_n = target - target.mean()
        pred_n = pred_n / pred_n.norm()
        target_n = target_n / target_n.norm()
        return (pred_n * target_n).sum()


    def spearman(self,
        target,
        pred,
        regularization="l2",
        regularization_strength=0.01,
    ):
        # fast_soft_sort uses 1-based indexing, divide by len to compute percentage of rank
        pred = soft_rank(
            pred,
            regularization=regularization,
            regularization_strength=regularization_strength,
        )
    #     print(pred)
        return self.corrcoef(target, pred / pred.shape[-1])
     
class CVPRLoss2(nn.Module):
    #kendall soft rank+tanh
    def __call__(self, pred, y):
        pred = pred.cpu()
        y = y.cpu()
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
        
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]

        tmp = soft_rank(
            preds.index_select(1, sa),
            regularization='l2',
            regularization_strength=0.01,
        )[0]-1
        
        tmp = tmp.cuda()

        score = torch.cat([((tmp[i:]-tmp[i-1]) * 10).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_softrank(nn.Module):
    #kendall soft rank+tanh
    def __call__(self, pred, y):
        pred = pred.cpu()
        y = y.cpu()
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
        
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]

        tmp = soft_rank(
            preds.index_select(1, sa),
            regularization='l2',
            regularization_strength=0.01,
        )[0]-1
        
        tmp = tmp.cuda()

        score = torch.cat([((tmp[i:]-tmp[i-1]) * 10).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_softsign(nn.Module):
    #kendall softsign
    def __call__(self, pred, y):
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)

    
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]
        tmp = preds.index_select(1, sa.int())[0]
        score = torch.cat([((tmp[i:]-tmp[i-1]))/(0.00001+torch.abs(tmp[i:]-tmp[i-1])) for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_tanh(nn.Module):
    # kendall tanh
    def __call__(self, pred, y):
        pred = pred
        y = y
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
    
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]
        tmp = preds.index_select(1, sa.int())[0]
        score = torch.cat([((tmp[i:]-tmp[i-1])*1).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_softrankf(nn.Module):
    #kendall soft rank+tanh
    def __call__(self, pred, y):
#         l0 = torch.abs(y-pred)

        pred = pred.cpu()
        y = y.cpu()
        l1 = 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]).reshape(1, -1)
        global mytest
        mytest.append([(1-l1).cpu().detach().numpy(),compute_metric(pred.detach().numpy(), y.detach().numpy())])
        return l1
        
        
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]

        tmp = soft_rank(
            preds.index_select(1, sa),
            regularization='l2',
            regularization_strength=0.01,
        )[0]-1
        
        tmp = tmp.cuda()
        score = torch.cat([((tmp[i:]-tmp[i-1]) * 1).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        global mytest1
        mytest1.append(tmp.cpu().detach().numpy())
        return score1
    
class CVPRLoss_softrankf1(nn.Module):
    #kendall soft rank+tanh
    def __call__(self, pred, y):
#         l0 = torch.abs(y-pred)

        pred = pred.cpu()
        y = y.cpu()
        l1 = 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]).reshape(1, -1)
#         global mytest
#         mytest.append([(1-l1).cpu().detach().numpy(),compute_metric(pred.detach().numpy(), y.detach().numpy())])
        return l1
        
        
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]

        tmp = soft_rank(
            preds.index_select(1, sa),
            regularization='l2',
            regularization_strength=0.01,
        )[0]-1
        
        tmp = tmp.cuda()
        score = torch.cat([((tmp[i:]-tmp[i-1]) * 1).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
#         global mytest1
#         mytest1.append(tmp.cpu().detach().numpy())
        return score1

In [26]:
def train(model, 
        train_dataset, 
        val_dataset, 
        verbose=20, 
        fold_=0,
        modelname='MLP_base',
        modelpath=r'./model'+'//',
        input='input',
        y='y',
        early_stop_round=60,
        debug=False):
    
    print(f'Model parameters count: {count_parameters(model)}')
    #数据加载
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=True,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE_TEST,
        shuffle=False,
        drop_last=False,
        pin_memory=True
    )
    print(f'train batch num: {len(train_loader)}')
    print(f'val batch num: {len(val_loader)}')
            
    # Optimizer
    optimizer = getattr(torch.optim, optim)(model.parameters(), lr=LR)
    # Scheduler
    num_warmup_steps = int(0.1 * EPOCHS * len(train_loader))
    num_training_steps = int(EPOCHS * len(train_loader))
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps, num_training_steps
    )
    print(f'optim: {optim}, lr: {LR}, warmup_steps: {num_warmup_steps}')
    
    print(f'early stopping round: {early_stop_round}\n')
    #train
    score_best=0
    bst_epoch=0
    first_epoch_eval=0
    for epoch in range(EPOCHS):
        if epoch > early_stop_round and (epoch - bst_epoch > early_stop_round):
            print(f'early stopping.')
            break
        
        model.train()
        model.zero_grad()
        start_time = time.time()

        avg_loss = 0
        for data in train_loader:
            pred = model(data[input].to(device))
#             print(pred.shape,data['y'].shape)

            loss = loss_fct(
                pred,
#                 data[y+'_quantile'].to(device),
                data[y].to(device),
            ).mean()

            loss.backward()
            avg_loss += loss.item() / len(train_loader)

            optimizer.step()
            scheduler.step()

            optimizer.zero_grad(set_to_none=True)

        #VAL
        model.eval()
        mae, avg_val_loss = 0, 0
        preds = []

        with torch.no_grad():
            for data in val_loader:
                pred = model(data[input].to(device))

                loss = loss_fct(
                    pred,
                    data[y].to(device)
                ).mean()

                avg_val_loss += loss.item() / len(val_loader)

                preds.append(pred.detach().cpu().numpy())

        preds = np.concatenate(preds, 0)
        if y=='y':
            mae = compute_metric(preds,val_dataset.df[target_cols].values).mean()
        else:
            mae = compute_metric(preds,val_dataset.df[[target_cols[int(y.replace('y',''))]]].values).mean()
        
        elapsed_time = time.time() - start_time
        if (epoch + 1) % verbose == 0:
            elapsed_time = elapsed_time * verbose
            lr = scheduler.get_last_lr()[0]
    #         lr=LR
            print(
                f"Epoch {epoch + 1:02d}/{ EPOCHS:02d} \t lr={lr:.1e}\t t={elapsed_time:.0f}s \t"
                f"loss={avg_loss:.4f}",
                end="\t",
            )

            if (epoch + 1 >= first_epoch_eval) or (epoch + 1 == EPOCHS):
                print(f"val_loss={avg_val_loss:.4f}\tcorr={mae:.4f}")
            else:
                print("")
                
        #保存最优模型
        if mae>score_best:
            bst = save_model_weights(model, modelpath, f'{modelname}_{fold_}.pt')
            score_best = mae
            bst_epoch = epoch
            if y=='y':
                bst_list = compute_metric(preds,val_dataset.df[target_cols].values)
            else:
                bst_list = compute_metric(preds,val_dataset.df[[target_cols[int(y.replace('y',''))]]].values).mean()
            bst_preds = preds
    print(f'best score {score_best}, best epoch: {bst_epoch}, {bst} ' )
    print(bst_list,'\n\n')
    del (val_loader, train_loader, loss, data, pred)
    gc.collect()
    torch.cuda.empty_cache()
    
    return bst_preds, bst_list


In [27]:
class CVPRModel(nn.Module):
    def __init__(
        self,
        input_dim=3,
        num_classes=8,
        time_step=12,
        bi=True
    ):
        super(CVPRModel,self).__init__()
        
        self.bi_num = 2 if bi else 1
        
        self.MLP = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )

        
        self.LSTM = nn.Sequential(
            nn.LSTM(256, 256, 
                    batch_first=True, 
                    bidirectional=bi, 
                    num_layers=1,
                    dropout=0)
                                 )
        
        self.LSTM1 = nn.LSTM(512, 256, 
                    batch_first=True, 
                    bidirectional=bi, 
                    num_layers=1,
                    dropout=0
                            )         
        
        
#         self.transformer = nn.TransformerEncoderLayer(
#             d_model=768, 
#             nhead=16, 
#             dropout=0.1,
#             dim_feedforward=768*4
#         )
        
        self.Logits = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(512*time_step, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        x1 = self.MLP(x)
        x2,(h0, c0) = self.LSTM(x1)
        x3,(h1, c1) = self.LSTM1(x2, (h0, c0))
        
        pred = self.Logits(x3)
        return pred

In [28]:
class CVPRModel(nn.Module):
    def __init__(
        self,
        input_dim=3,
        num_classes=8,
        time_step=12,
        bi=True
    ):
        super(CVPRModel,self).__init__()
        
        self.bi_num = 2 if bi else 1
        
        self.MLP = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )

        self.LSTM = nn.Sequential(
            nn.LSTM(256, 256, 
                    batch_first=True, 
                    bidirectional=bi, 
                    num_layers=1,
                    dropout=0)
                                 )
        
        self.LSTM1 = nn.LSTM(512, 256, 
                    batch_first=True, 
                    bidirectional=bi, 
                    num_layers=1,
                    dropout=0
                            ) 
        
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=256, 
            nhead=2, 
            dropout=0,
            dim_feedforward=256*4
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer=self.encoder_layer, 
            num_layers=1
        )
        
        
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=256, 
            nhead=2,
            dropout=0,
            dim_feedforward=256*4
        )      
        self.decoder = nn.TransformerDecoder(
            decoder_layer=self.decoder_layer, 
            num_layers=1
        )
        
        
        self.Logits = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(768*time_step, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        x1 = self.MLP(x)
        
        x2,(h0, c0) = self.LSTM(x1)
        x3,(h1, c1) = self.LSTM1(x2, (h0, c0))
        
        enc = self.encoder(x1)
        dec = self.decoder(x1, enc)
        
        c = torch.cat([x3, dec],-1)
        
        pred = self.Logits(c)
        return pred

In [29]:
class CVPRModel(nn.Module):
    def __init__(
        self,
        input_dim=3,
        num_classes=8,
        time_step=12,
        bi=True
    ):
        super(CVPRModel,self).__init__()
        
        self.bi_num = 2 if bi else 1
        
        self.MLP = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )

        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=256, 
            nhead=2, 
            dropout=0,
            dim_feedforward=256*4
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer=self.encoder_layer, 
            num_layers=1
        )
        
        
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=256, 
            nhead=2,
            dropout=0,
            dim_feedforward=256*4
        )      
        self.decoder = nn.TransformerDecoder(
            decoder_layer=self.decoder_layer, 
            num_layers=1
        )
        
        
        self.Logits = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(256*time_step, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        x1 = self.MLP(x)
        enc = self.encoder(x1)
        dec = self.decoder(x1, enc)
        
        pred = self.Logits(dec)
        return pred

In [30]:
#参数设置
seed = 666
BATCH_SIZE = 384
BATCH_SIZE_TEST = 128
EPOCHS = 300
LR = 0.0001
optim = "Adam"
device = "cuda" if torch.cuda.is_available() else "cpu"
modelpath = r'./model'+'//'

In [31]:
a = torch.rand(128,37,1).to(device)
net = CVPRModel(input_dim=1,
                num_classes=len(target_cols),
                bi=True,
                time_step=37
               ).to(device)
b = net(a)
b.shape

torch.Size([128, 8])

In [32]:
b[0].shape

torch.Size([8])

In [33]:
b[1].shape

torch.Size([8])

In [34]:
b.flatten(1).shape

torch.Size([128, 8])

In [35]:
del a,b,net;
gc.collect()
torch.cuda.empty_cache()

In [36]:
mytest = []
mytest1 = []

In [37]:
loss_fct = CVPRLoss_softrankf1()
loss_fct = CVPRLoss_tanh()
index = '_t'
seed_everything(seed)
modelname = 'LSTM1_step37_base1_test'

model = CVPRModel(input_dim=1,
                num_classes=8,
                bi=True,
                time_step=37
               ).to(device)

preds,_ = train(model, 
            train_dataset, 
            val_dataset, 
            verbose=20, 
            fold_=index,
            modelname=modelname,
            modelpath=modelpath,
            input='input',
            y='y',
            debug=False
             )

Model parameters count: 25548808
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.0001, warmup_steps: 30
early stopping round: 60

Epoch 20/300 	 lr=6.7e-05	 t=9s 	loss=0.8340	val_loss=0.8276	corr=0.3698
Epoch 40/300 	 lr=9.6e-05	 t=10s 	loss=0.2789	val_loss=0.3084	corr=0.7066
Epoch 60/300 	 lr=8.9e-05	 t=10s 	loss=0.1878	val_loss=0.2339	corr=0.7753
Epoch 80/300 	 lr=8.1e-05	 t=9s 	loss=0.1370	val_loss=0.2355	corr=0.7735
Epoch 100/300 	 lr=7.4e-05	 t=9s 	loss=0.0905	val_loss=0.2319	corr=0.7734
Epoch 120/300 	 lr=6.7e-05	 t=9s 	loss=0.0630	val_loss=0.2309	corr=0.7754
early stopping.
best score 0.7839898989898991, best epoch: 68, 
 -> Save weights to ./model//LSTM1_step37_base1_test__t.pt
 
[0.30868687 0.86868687 0.88848485 0.95111111 0.87393939 0.68080808
 0.89737374 0.80282828] 




In [347]:
len(mytest)

404

In [348]:
retf = []
for i in mytest:
    tmp0,tmp1 = i
    tr=list(tmp0[0])+list(tmp1)
    retf.append(tr)
retf = pd.DataFrame(retf,columns=target_cols+[x+'_loss' for x in target_cols])

In [349]:
for c in target_cols:
    print(c, np.abs(retf[c]-retf[c+'_loss']).mean()*100)

cplfw_rank 1.147863385396876
market1501_rank 1.6789394379490759
dukemtmc_rank 1.963100413137639
msmt17_rank 2.1124751344489137
veri_rank 1.5402529562518859
vehicleid_rank 1.424956932201814
veriwild_rank 2.2069543952296256
sop_rank 1.6854778987706704


In [350]:
for c in target_cols:
    print(c, np.abs(retf[c]-retf[c+'_loss']).mean()*100)

cplfw_rank 1.147863385396876
market1501_rank 1.6789394379490759
dukemtmc_rank 1.963100413137639
msmt17_rank 2.1124751344489137
veri_rank 1.5402529562518859
vehicleid_rank 1.424956932201814
veriwild_rank 2.2069543952296256
sop_rank 1.6854778987706704


In [351]:
#  0.7891414141414143,
#  array([0.33454545, 0.87070707, 0.88929293, 0.95232323, 0.88565657,
#         0.67111111, 0.90868687, 0.80080808]))
# 0.788939 [0.33737374 0.86828283 0.88969697 0.95838384 0.87838384 0.66787879 0.91555556 0.7959596 ] 

In [352]:
# 512 多头4，4 0.7927 [0.35515152 0.87232323 0.89050505 0.96484848 0.87676768 0.66343434  0.9159596  0.80282828] 
# 多头1，1 [0.34222222 0.87555556 0.89575758 0.96242424 0.8759596  0.66383838 0.91111111 0.80646465] 
# 多头2,2 0.7964646464646465,[0.37737374 0.87434343 0.89333333 0.9640404  0.87434343 0.66545455 0.91717172 0.80565657]
# 多头4,4 [0.34424242 0.87717172 0.89333333 0.96606061 0.88363636 0.65656566 0.90989899 0.80363636] 

In [353]:
# [0.36686869 0.87717172 0.89212121 0.95636364 0.8840404  0.66141414
#  0.91151515 0.80525253] 
# [0.34505051 0.87515152 0.89616162 0.96282828 0.88686869 0.66424242
#  0.91474747 0.79191919] 
# [0.35393939 0.87151515 0.89171717 0.96121212 0.8840404  0.66343434
#  0.91191919 0.8040404 ] 

In [354]:
# 2层自带lstm 0.7911616161616162 
# 1层的输出作为2层的h和c 0.7920707070707071
# 2层手写，不初始化 0.7911616161616162 
# 3层手写，输出作为输入 0.7911111111111113 
# 2自带*2自带，输出作为输入 0.7875757575757577 
# 1层输出作为2层输入，最后2层输出cat 0.7894444444444446
# 1层输出作为2层输入，最后2层输出相乘 0.7885858585858587
# 1层的输出作为2层的h和c,加一层gru cat 0.7894949494949497 
# 1层的输出作为2层的h和c,加一层gru*1层输出 cat 0.7886868686868689
# 1层的输出作为2层的h和c,加2层gru cat 0.7913131313131314 
# 1层的输出作为2层的h和c,加2层gru*1层输出 cat2层输出 0.7915151515151516 
# 1层的输出作为2层的h和c，2层的输出transformer然后乘以2层的输出，0.7899494949494951
# 1层的输出作为2层的h和c,Tanh 0.7893939393939395 
# 1层的输出作为2层的h和c,relu 0.7862121212121214 
# 1层的输出作为2层的h和c,fc出去，0.7872222222222224 

In [355]:
# index = '_t'
# modelname = 'LSTM1_step37_base1_test'
# scoref = []
# for i in range(8):
#     seed_everything(seed)
#     model = CVPRModel(input_dim=1,
#                     num_classes=1,
#                     bi=True,
#                     time_step=37
#                    ).to(device)
#     preds,_ = train(model, 
#                 train_dataset, 
#                 val_dataset, 
#                 verbose=20, 
#                 fold_=index,
#                 modelname=modelname,
#                 modelpath=modelpath,
#                 input='input',
#                 y=f'y{i}',
#                 debug=False
#                  )
#     scoref.append(_)
# np.mean(scoref)

In [356]:
# [0.33171717 0.86222222 0.88888889 0.94222222 0.8630303  0.66909091 0.9030303  0.79757576]

In [38]:
# K折训练，同时
# loss_fct = CVPRLoss_softrankf()
loss_fct = CVPRLoss_tanh()
k=5
scoref = []
skf = KFold(n_splits=k, shuffle=False)
for index, (train_index, test_index) in enumerate(skf.split(train_df)):   
    print(f'FOLD {index}')
    train0 = train_df.iloc[train_index]
    val0 = train_df.iloc[test_index]   
    train_dataset = MyDataset(train0, use_cols, target_cols)
    val_dataset = MyDataset(val0, use_cols, target_cols)
    print(f'train size: {len(train0)}, val size: {len(val0)}')

    modelname = f'transformer_encoder_decoder_hardtanh'
    seed_everything(seed)
    model = CVPRModel(input_dim=1,
                    num_classes=8,
                    bi=True,
                    time_step=37
                   ).to(device)
    preds,_ = train(model, 
                train_dataset, 
                val_dataset, 
                verbose=100, 
                fold_=index,
                modelname=modelname,
                modelpath=modelpath,
                input='input',
                y='y',
                debug=False
                 )
    scoref.append(_)
scoreff = scoref
np.round(np.array(scoreff).mean(1).mean(), 5), [round(x, 5) for x in np.array(scoreff).mean(0)]

FOLD 0
train size: 400, val size: 100
Model parameters count: 25548808
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.0001, warmup_steps: 30
early stopping round: 60

Epoch 100/300 	 lr=7.4e-05	 t=47s 	loss=0.0923	val_loss=0.2270	corr=0.7798
early stopping.
best score 0.785858585858586, best epoch: 66, 
 -> Save weights to ./model//transformer_encoder_decoder_hardtanh_0.pt
 
[0.28       0.89737374 0.89292929 0.94909091 0.88040404 0.6820202
 0.91959596 0.78545455] 


FOLD 1
train size: 400, val size: 100
Model parameters count: 25548808
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.0001, warmup_steps: 30
early stopping round: 60

Epoch 100/300 	 lr=7.4e-05	 t=53s 	loss=0.0923	val_loss=0.2323	corr=0.7742
early stopping.
best score 0.7763131313131314, best epoch: 82, 
 -> Save weights to ./model//transformer_encoder_decoder_hardtanh_1.pt
 
[0.27717172 0.86868687 0.89979798 0.95393939 0.88646465 0.64080808
 0.88808081 0.79555556] 


FOLD 2
train size: 400, val size: 100
Mo

(0.77898,
 [0.25519, 0.86675, 0.89358, 0.952, 0.89236, 0.66723, 0.91265, 0.79208])

In [39]:
0.77939,[0.25301, 0.86158, 0.89915, 0.96024, 0.89277, 0.66764, 0.91451, 0.78626]

(0.77939,
 [0.25301, 0.86158, 0.89915, 0.96024, 0.89277, 0.66764, 0.91451, 0.78626])

In [40]:
np.round(np.array(scoreff).mean(1).mean(), 5), [round(x, 5) for x in np.array(scoreff).mean(0)]

(0.77898,
 [0.25519, 0.86675, 0.89358, 0.952, 0.89236, 0.66723, 0.91265, 0.79208])

In [41]:
# 多头384 (0.78, [0.2682, 0.86602, 0.89737, 0.95741, 0.88905, 0.66335, 0.91281, 0.78578])

In [42]:
# K折训练，目标分别
# scoreff = []
# for target in range(8):
#     k=5
#     scoref = []
#     skf = KFold(n_splits=k, shuffle=False)
#     for index, (train_index, test_index) in enumerate(skf.split(train_df)):   
#         print(f'TARGET {target} FOLD {index}')
#         train0 = train_df.iloc[train_index]
#         val0 = train_df.iloc[test_index]   
#         train_dataset = MyDataset(train0, use_cols, target_cols)
#         val_dataset = MyDataset(val0, use_cols, target_cols)
#         print(f'train size: {len(train0)}, val size: {len(val0)}')

#         modelname = f'LSTM_2layer_step37_kendall_tanh_target{target}'
#         seed_everything(seed)
#         model = CVPRModel(input_dim=1,
#                         num_classes=1,
#                         bi=True,
#                         time_step=37
#                        ).to(device)
#         preds,_ = train(model, 
#                     train_dataset, 
#                     val_dataset, 
#                     verbose=100, 
#                     fold_=index,
#                     modelname=modelname,
#                     modelpath=modelpath,
#                     input='input',
#                     y=f'y{target}',
#                     debug=False
#                      )
#         scoref.append(_)
#     scoreff.append(scoref)

In [43]:
np.round(np.array(scoreff).mean(1).mean(),5),[round(x,5) for x in np.array(scoreff).mean(1)]

(0.77898, [0.78586, 0.77631, 0.7854, 0.76015, 0.78717])

In [44]:
(0.78795,
 [0.26836, 0.87111, 0.90545, 0.96509, 0.89745, 0.67507, 0.91895, 0.8021])

(0.78795,
 [0.26836, 0.87111, 0.90545, 0.96509, 0.89745, 0.67507, 0.91895, 0.8021])

In [45]:
#加载test
with open('./data/CVPR_2022_NAS_Track2_test.json', 'r') as f:
    test_data = json.load(f)
test_data['arch99997']

{'cplfw_rank': 0,
 'market1501_rank': 0,
 'dukemtmc_rank': 0,
 'msmt17_rank': 0,
 'veri_rank': 0,
 'vehicleid_rank': 0,
 'veriwild_rank': 0,
 'sop_rank': 0,
 'arch': 'j121221121221221311331321121221000000'}

In [46]:
test_df = get_df(test_data)
get_step(test_df)

In [47]:
test_df[[x+'_quantile' for x in target_cols]] = 0

In [48]:
def pred_cuda(test_df, model):
    #获得预测
    test_dataset = MyDataset(test_df, use_cols, target_cols, 0)
    test_loader = DataLoader(
        test_dataset,
        batch_size=1024*2,
        shuffle=False,
        drop_last=False,
        pin_memory=True
    )

    preds = []
    with torch.no_grad():
        for data in tqdm(test_loader):
            pred = model(data['input'].to(device))
            preds.append(pred.detach().cpu().numpy())
    preds = np.concatenate(preds, 0)
    print(preds.shape)
    return preds
        
def predict_sig(test_df, k=5):
    for target in range(8):
        
        modelname = f'LSTM_2layer_step37_kendall_tanh_target{target}'
        print(f'Model {modelname}')
        
        cols = []
        for fold_ in range(k):
            model = CVPRModel(input_dim=1,
                        num_classes=1,
                        bi=True,
                        time_step=37
                       ).to(device)
            model.load_state_dict(torch.load(modelpath+f'{modelname}_{fold_}.pt'))

            pred_ = pred_cuda(test_df, model)
            tmp_c = f'{target_cols[target]}_{fold_}'
            test_df[tmp_c] = pred_
            cols.append(tmp_c)
            print(f'Done {tmp_c}')
        print(cols)
        test_df[cols] = test_df[cols].rank()
        test_df[target_cols[target]] = test_df[cols].mean(axis=1).rank()
        
def predict_all(test_df, k=5):
        
    modelname = f'transformer_encoder_decoder_hardtanh'
    print(f'Model {modelname}')
    
    k_list = [0,1,2,3,4]

    cols = []
    for fold_ in range(k):
        if fold_ not in k_list:
            continue
        model = CVPRModel(input_dim=1,
                    num_classes=8,
                    bi=True,
                    time_step=37
                   ).to(device)
        model.load_state_dict(torch.load(modelpath+f'{modelname}_{fold_}.pt'))

        pred_ = pred_cuda(test_df, model)
        tmp_c = [f'{target}_{fold_}' for target in target_cols]
        test_df[tmp_c] = pred_
        cols += tmp_c
    #get rank
    print(cols)
    test_df[cols] = test_df[cols].rank()
    for c in target_cols:
        test_df[c] = test_df[[f'{c}_{fold_}' for fold_ in k_list]].mean(axis=1)
        test_df[c] = test_df[c].rank()

In [49]:
k, modelname

(5, 'transformer_encoder_decoder_hardtanh')

In [50]:
# LSTM_2layer_step37_kendall_8all_softrankloss
# LSTM_2layer_step37_kendall_8all_softsign
# LSTM_2layer_step37_kendall_8all_tanh


In [51]:
k=5

In [52]:
model_dt = predict_all(test_df, k=k)

Model transformer_encoder_decoder_hardtanh


C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/49 [00:00<?, ?it/s]

(99500, 8)


  0%|          | 0/49 [00:00<?, ?it/s]

...

(99500, 8)


  0%|          | 0/49 [00:00<?, ?it/s]

(99500, 8)


  0%|          | 0/49 [00:00<?, ?it/s]

(99500, 8)
['cplfw_rank_0', 'market1501_rank_0', 'dukemtmc_rank_0', 'msmt17_rank_0', 'veri_rank_0', 'vehicleid_rank_0', 'veriwild_rank_0', 'sop_rank_0', 'cplfw_rank_1', 'market1501_rank_1', 'dukemtmc_rank_1', 'msmt17_rank_1', 'veri_rank_1', 'vehicleid_rank_1', 'veriwild_rank_1', 'sop_rank_1', 'cplfw_rank_2', 'market1501_rank_2', 'dukemtmc_rank_2', 'msmt17_rank_2', 'veri_rank_2', 'vehicleid_rank_2', 'veriwild_rank_2', 'sop_rank_2', 'cplfw_rank_3', 'market1501_rank_3', 'dukemtmc_rank_3', 'msmt17_rank_3', 'veri_rank_3', 'vehicleid_rank_3', 'veriwild_rank_3', 'sop_rank_3', 'cplfw_rank_4', 'market1501_rank_4', 'dukemtmc_rank_4', 'msmt17_rank_4', 'veri_rank_4', 'vehicleid_rank_4', 'veriwild_rank_4', 'sop_rank_4']


In [53]:
test_df[target_cols].describe()

,cplfw_rank,market1501_rank,dukemtmc_rank,msmt17_rank,veri_rank,vehicleid_rank,veriwild_rank,sop_rank
count,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000
mean,49750.500000,49750.500000,49750.500000,49750.500000,49750.500000,49750.500000,49750.500000,49750.500000
std,28723.320228,28723.320228,28723.320228,28723.320228,28723.320228,28723.320228,28723.320228,28723.320228
min,1.000000,1.000000,1.000000,1.500000,1.000000,1.000000,1.000000,1.000000
25%,24875.000000,24875.750000,24875.750000,24875.750000,24875.750000,24875.750000,24875.750000,24875.750000
50%,49750.500000,49750.750000,49750.500000,49750.500000,49750.500000,49750.500000,49750.500000,49750.500000
75%,74625.250000,74625.250000,74625.250000,74625.250000,74625.375000,74625.375000,74625.250000,74625.375000
max,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000


In [54]:
test_df[target_cols] = test_df[target_cols].astype(int)-1

In [55]:
test_df[target_cols].describe()

,cplfw_rank,market1501_rank,dukemtmc_rank,msmt17_rank,veri_rank,vehicleid_rank,veriwild_rank,sop_rank
count,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000
mean,49749.419427,49749.418402,49749.419116,49749.418804,49749.418382,49749.419226,49749.417889,49749.419749
std,28723.319720,28723.320380,28723.318907,28723.321470,28723.320690,28723.320862,28723.320015,28723.320291
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24874.000000,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000
50%,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000
75%,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000
max,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000


In [56]:
modelname

'transformer_encoder_decoder_hardtanh'

In [57]:
modelname = 'transformer_encoder_decoder_hardtanh'

In [58]:
#输出
for i in tqdm(test_df[['id']+target_cols].values):
    id_ = i[0]
    for k,v in enumerate(target_cols):
        k += 1
        test_data[id_][v] = i[k]
        
with open(f'./sub/{modelname}_20220509.json', 'w') as f:
    json.dump(test_data, f)

C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/99500 [00:00<?, ?it/s]

In [59]:
modelname

'transformer_encoder_decoder_hardtanh'

In [60]:
model

CVPRModel(
  (MLP): Sequential(
    (0): Linear(in_features=1, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
  )
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
    )
    (linear1): Linear(in_features=256, out_features=1024, bias=True)
    (dropout): Dropout(p=0, inplace=False)
    (linear2): Linear(in_features=1024, out_features=256, bias=True)
    (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0, inplace=False)
    (dropout2): Dropout(p=0, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
      